# OilyGiant

## Visão do Projeto

Você trabalha na empresa de mineração OilyGiant. Sua tarefa é encontrar os melhores lugares para o **desenvolvimento de novos poços de petróleo**.

Para concluir essa tarefa, você vai precisar executar as seguintes etapas:

- Ler os arquivos com parâmetros coletados de poços de petróleo na região selecionada: a quantidade de petróleo e o volume de reservas;

- Construir um modelo para predizer o volume de reservas em novos poços;

- Escolher os poços de petróleo que têm os maiores valores estimados;

- Escolher a região com o maior lucro total para os poços de petróleo selecionados.

Você tem dados sobre amostras de petróleo de três regiões. Os parâmetros de cada poço de petróleo na região já são conhecidos. Construa um modelo que te ajudará a escolher a região com a margem de lucro mais alta. Utilize a técnica de Bootstrapping para analisar lucro potencial e riscos.

---

# 1. INICIALIZAÇÃO

# 1.1 Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np

# Divisào do dataset
from sklearn.model_selection import train_test_split

# Modelo
from sklearn.linear_model import LinearRegression

# Metricas
from sklearn.metrics import mean_squared_error

# Pipeline
from sklearn.pipeline import Pipeline

# Column Transformer
from sklearn.compose import ColumnTransformer

# Pre processamento
from sklearn.preprocessing import StandardScaler

## 1.2 Carregando Dataset

In [ ]:
# Carregue os arquivos de dados em diferentes DataFrames

# Local Path
local_geo_0_path = "geo_data_0.csv"
local_geo_1_path = "geo_data_1.csv"
local_geo_2_path = "geo_data_2.csv"

# Cloud Path
geo_0_path = "/datasets/geo_data_0.csv"
geo_1_path = "/datasets/geo_data_1.csv"
geo_2_path = "/datasets/geo_data_2.csv"

try:
    geo_0 = pd.read_csv(local_geo_0_path, sep=",")
    geo_1 = pd.read_csv(local_geo_1_path, sep=",")
    geo_2 = pd.read_csv(local_geo_2_path, sep=",")

except FileNotFoundError:
    print(
        f"O arquivo CSV não foi encontrado em {local_geo_0_path}. Tentando o caminho {geo_0_path}..."
    )

    print(
        f"O arquivo CSV não foi encontrado em {local_geo_1_path}. Tentando o caminho {geo_1_path}..."
    )

    print(
        f"O arquivo CSV não foi encontrado em {local_geo_2_path}. Tentando o caminho {geo_2_path}..."
    )

    try:
        geo_0 = pd.read_csv(geo_0_path, sep=",")
        geo_1 = pd.read_csv(geo_1_path, sep=",")
        geo_2 = pd.read_csv(geo_2_path, sep=",")

    except FileNotFoundError:
        print(
            f"O arquivo CSV não foi encontrado em {geo_0_path}. Nenhum arquivo encontrado."
        )

        print(
            f"O arquivo CSV não foi encontrado em {geo_1_path}. Nenhum arquivo encontrado."
        )

        print(
            f"O arquivo CSV não foi encontrado em {geo_2_path}. Nenhum arquivo encontrado."
        )

In [ ]:
# Checando se DataFrame foi importado
geo_0.head()

In [ ]:
# Checando se DataFrame foi importado
geo_1.head()

In [ ]:
# Checando se DataFrame foi importado
geo_2.head()

## 1.3 Analisando os Dados

#### 1.3.1 Dataset: geo_0

In [ ]:
# dtypes dos dados
geo_0.dtypes

In [ ]:
# Info dos dados
geo_0.info()

In [ ]:
# Checando dados faltosos
geo_0.isna().sum()

In [ ]:
# Checando duplicados
geo_0.duplicated().sum()

In [ ]:
# Describe dos dados
geo_0.describe()

- O dataset geo_0 não apresenta nenhuma inconformidade. Os dados estão prontos para serem utilizados.

#### 1.3.2 Dataset: geo_1

In [ ]:
# dtypes dos dados
geo_1.dtypes

In [ ]:
# Info dos dados
geo_1.info()

In [ ]:
# Checando dados faltosos
geo_1.isna().sum()

In [ ]:
# Checando duplicados
geo_1.duplicated().sum()

In [ ]:
# Describe dos dados
geo_1.describe()

- O dataset geo_1 não apresenta nenhuma inconformidade. Os dados estão prontos para serem utilizados.

#### 1.3.3 Dataset: geo_2

In [ ]:
# dtypes dos dados
geo_2.dtypes

In [ ]:
# Info dos dados
geo_2.info()

In [ ]:
# Checando dados faltosos
geo_2.isna().sum()

In [ ]:
# Checando duplicados
geo_2.duplicated().sum()

In [ ]:
# Describe dos dados
geo_2.describe()

- O dataset geo_2 não apresenta nenhuma inconformidade. Os dados estão prontos para serem utilizados.

## 2. PREPARAÇÃO DO MODELO

In [ ]:
def preprocess_train_model(df):
    # Separando o dataset em traino e validação
    train_data, valid_data = train_test_split(df, test_size=0.25, random_state=12345)

    # Definir as features e target
    features_train, target_train = (
        train_data.drop(["product"], axis=1),
        train_data["product"],
    )
    features_valid, target_valid = (
        valid_data.drop(["product"], axis=1),
        valid_data["product"],
    )

    # Criando pipeline de pre-processamento.
    numeric_features = ["f0", "f1", "f2"]
    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])
    preprocessor = ColumnTransformer(
        transformers=[("num", numeric_transformer, numeric_features)]
    )

    model = Pipeline(
        steps=[("preprocessor", preprocessor), ("regressor", LinearRegression())]
    )

    # Treinar o modelo
    model.fit(features_train, target_train)

    # Predições no conjunto de validação
    predictions = model.predict(features_valid)

    # Salvar predições
    results_df = pd.DataFrame({"true_values": target_valid, "predicted": predictions})

    # Métricas de avaliação
    avg_predicted_volume = predictions.mean()
    rmse = mean_squared_error(target_valid, predictions, squared=False)

    print(f"Volume Médio Previsto: {avg_predicted_volume:.4f}")
    print(f"REQM: {rmse:.4f}")

    return results_df

In [ ]:
# Rodando o modelo para o dataset: geo_0
results_0 = preprocess_train_model(geo_0)
results_0

- O volume médio de produção para a região 0 é de 92.5926 milhares de barris

- Podemos ver que para o conjunto de dados de geo_0, temos um Root Mean Squared Error (REQM) igual à 37,5794. Isso indica que, em média, as predições do modelo têm um devio de aproximadamente 37,5794 milhares de barris em relação aos valores reais.

Lembrando que quanto menor o valor do REQM, melhor o desempenho do modelo. Nesse caso, seria desejável ter um REQM mais baixo para indicar uma precisão maior nas predições do modelo de regressão linear.

In [ ]:
# Rodando o modelo para o dataset: geo_1
results_1 = preprocess_train_model(geo_1)
results_1

- O volume médio de produção para a região 1 é de 68.7285 milhares de barris

- Podemos ver que para o conjunto de dados de geo_01 temos um Root Mean Squared Error (REQM) igual à 0,8931. Isso indica que, em média, as predições do modelo têm um devio de aproximadamente 0,8931 milhares de barris em relação aos valores reais.

Ou seja, o modelo prevê com muita precisão o volume de reservas no poço de petróleo.

In [ ]:
# Rodando o modelo para o dataset: geo_2
results_2 = preprocess_train_model(geo_2)
results_2

- O volume médio de produção para a região 2 é de 94.9650 milhares de barris

- Podemos ver que para o conjunto de dados de geo_2, temos um Root Mean Squared Error (REQM) igual à 40,0297. Isso indica que, em média, as predições do modelo têm um devio de aproximadamente 40,0297 milhares de barris em relação aos valores reais.

Dentre os 3 conjuntos de dados (geo_0, geo_1 e geo_2), esse conjunto teve o maior desvio, em média, dos valores reais. 

## 3. CALCULANDO O LUCRO

In [ ]:
# Valores necessários para calculo
investment = 100000000
revenue_per_unit = 4500
required_units = 111.1

# Calculo do volume médio necessário para evitar prejuízos
volume_needed = investment / (revenue_per_unit * required_units)

In [ ]:
# Valor médio de cada região
avg_volume_0 = geo_0["product"].mean()
avg_volume_1 = geo_1["product"].mean()
avg_volume_2 = geo_2["product"].mean()

In [ ]:
print(f"Volume Médio Necessário: {volume_needed:.4f}")
print(f"Volume Médio da Região 0: {avg_volume_0:.4f}")
print(f"Volume Médio da Região 1: {avg_volume_1:.4f}")
print(f"Volume Médio da Região 2: {avg_volume_2:.4f}")

A região que tem o maior volume de produção é a região 2, seguida da região 0 e por fim a região 1.

## 4. FUNÇÃO PARA CALCULAR O LUCRO DE UM CONJUNTO DE DADOS

In [ ]:
def calculate_profit(selected_wells, predictions):
    # Selecionando os 200 poços com os valores mais altos previstos
    selected_wells = predictions.nlargest(200, "predicted")

    # Soma do volume de reservas
    target_volume = selected_wells["true_values"].sum()

    # Calculo do lucro
    profit = target_volume * revenue_per_unit - investment

    return profit

In [ ]:
# Calcular lucro potencial para região 0
profit_0 = calculate_profit(geo_0, results_0)
profit_0

In [ ]:
# Calcular lucro potencial para região 1
profit_1 = calculate_profit(geo_1, results_1)
profit_1

In [ ]:
# Calcular lucro potencial para região 2
profit_2 = calculate_profit(geo_2, results_2)
profit_2

In [ ]:
# Apresentar conclusões sobre lucro potencial
print(f"Lucro Potencial Região 0: {profit_0:.2f}")
print(f"Lucro Potencial Região 1: {profit_1:.2f}")
print(f"Lucro Potencial Região 2: {profit_2:.2f}")

- Região 0: O lucro potencial estimado para a região 0 é de aproximadamente 33.2 milhões.
- Região 1: Para a região 1, o lucro potencial é cerca de 24.2 milhões.
- Região 2: O lucro potencial estimado para a região 2 é em torno de 27.1 milhões.

### 5. CALCULANDO RISCOS E LUCROS PARA CADA REGIÃO

In [ ]:
def calculate_bootstrap(data, predictions, iterations=1000):
    state = np.random.RandomState(12345)
    profits = []

    for i in range(iterations):
        # Amostrar com reposição
        sampled_size = min(len(data), len(predictions))
        sampled_data = data.sample(n=sampled_size, replace=True, random_state=state)

        # Verificar se os tamanhos dos DataFrames são iguais
        if len(sampled_data) == len(predictions):
            # Usar índices numéricos com iloc
            sampled_data.reset_index(drop=True, inplace=True)
            sampled_predictions = predictions.reset_index(drop=True).iloc[
                sampled_data.index
            ]

            # Calcular lucro potencial para a amostra
            profit = calculate_profit(sampled_data, sampled_predictions)
            profits.append(profit)
        else:
            print("Tamanhos diferentes entre sampled_data e predictions.")

    mean_profit = np.mean(profits)
    ci_lower, ci_upper = np.percentile(profits, [2.5, 97.5])
    risk_of_loss = np.mean(np.array(profits) < 0) * 100

    return mean_profit, ci_lower, ci_upper, risk_of_loss

In [ ]:
# Calcular para cada região
bootstrap_results_0 = calculate_bootstrap(geo_0, results_0)
bootstrap_results_1 = calculate_bootstrap(geo_1, results_1)
bootstrap_results_2 = calculate_bootstrap(geo_2, results_2)

In [ ]:
# Apresentar conclusões sobre riscos e lucro
print("Região 0:")
print(f"Lucro Médio: {bootstrap_results_0[0]:.2f}")
print(
    f"Intervalo de Confiança (95%): ({bootstrap_results_0[1]:.2f}, {bootstrap_results_0[2]:.2f})"
)
print(f"Risco de Prejuízo: {bootstrap_results_0[3]:.2f}%\n\n")

print("Região 1:")
print(f"Lucro Médio: {bootstrap_results_1[0]:.2f}")
print(
    f"Intervalo de Confiança (95%): ({bootstrap_results_1[1]:.2f}, {bootstrap_results_1[2]:.2f})"
)
print(f"Risco de Prejuízo: {bootstrap_results_1[3]:.2f}%\n\n")

print("Região 2:")
print(f"Lucro Médio: {bootstrap_results_2[0]:.2f}")
print(
    f"Intervalo de Confiança (95%): ({bootstrap_results_2[1]:.2f}, {bootstrap_results_2[2]:.2f})"
)
print(f"Risco de Prejuízo: {bootstrap_results_2[3]:.2f}%")

Com base nos resultados obtidos para cada região usando a técnica de bootstrap, podemos fazer as seguintes conclusões:

- Lucro Médio:

   - Região 0: O lucro médio é de aproximadamente 33.2 milhões.
   - Região 1: A média do lucro é cerca de 24.2 milhões.
   - Região 2: O lucro médio é em torno de 27.1 milhões.

- Intervalo de Confiança (95%):

    - Para todas as regiões, o intervalo de confiança (95%) é bastante restrito, indicando uma precisão considerável nos resultados do bootstrap. Isso significa que, com 95% de confiança, espera-se que o verdadeiro valor do lucro esteja dentro do intervalo fornecido.

- Risco de Prejuízo:

Para todas as regiões, o risco de prejuízo é relatado como 0.00%. Isso indica que, com base nas amostras bootstrap, não foram observados cenários em que o lucro foi negativo.

## 6. CONCLUSÃO

Com base nos cálculos de lucro potencial para cada região, podemos tirar as seguintes conclusões:

- Lucro Potencial:

    - Região 0: O lucro potencial estimado para a região 0 é de aproximadamente 33.2 milhões.
    - Região 1: Para a região 1, o lucro potencial é cerca de 24.2 milhões.
    - Região 2: O lucro potencial estimado para a região 2 é em torno de 27.1 milhões.

- Comparação com os Resultados do Bootstrap:

    Os valores de lucro potencial obtidos diretamente do cálculo para cada região são consistentes com os resultados obtidos através da técnica de bootstrap. Ambas as abordagens indicam desempenho positivo em termos de lucro.

- Ranking de Regiões em Termos de Lucro Potencial:

    - A região 0 apresenta o maior lucro potencial entre as três regiões.
    - A região 2 possui um lucro potencial ligeiramente superior à região 1.

Portanto, com base nos cálculos de lucro potencial, a região 0 é a mais promissora em termos financeiros, seguida pela região 2 e, por fim, a região 1. Essas conclusões são fundamentadas na suposição de que os cálculos de lucro potencial são representativos do desempenho real das regiões em relação aos poços de petróleo.